In [77]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
import pandas as pd
from joblib import Parallel, delayed

load_dotenv("./.env")

client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret), requests_timeout=45)

In [109]:
df = pd.read_csv('tracks6.csv')
list_songs = df['0'].dropna().values.tolist()
batch_ids = [list_songs[i:i+99] for i in range(0, len(list_songs), 99)]

In [110]:
def getFeatures(id):
    try: return sp.audio_features(id)
    except: return

features = Parallel(
    n_jobs=50
)(
    delayed(getFeatures)(i) for i in batch_ids
)

In [111]:
features2 = [i for i in features if i != None]
flattened = [b for b in features2 for b in b if b]

In [ ]:
df2 = pd.DataFrame(flattened).drop_duplicates()
# df2.to_csv('features.csv', index=False)